# cellbody2neuron
Using Jasper's idea to predict parent neurons for nuclei, Thank you so much, Jasper.

In [1]:
# https://www.j-focus.jp/user_guide/ug0004040000/
# sbatch

In [2]:
# libraries 1
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import argparse

from cloudvolume import CloudVolume, view, Bbox
import fill_voids
from taskqueue import TaskQueue, queueable
from functools import partial

In [3]:
# libraries 2
# %pwd
sys.path.append(os.path.abspath("../segmentation"))
# to import rootID_lookup and authentication_utils like below
import rootID_lookup as IDlook
import authentication_utils as auth

In [ ]:
parser = argparse.ArgumentParser(description='get segIDs of parent neurons from csv files') 
parser.add_argument('-c', '--choose', help='specify the numer of pixels randomly chosen to get segID of parent neuron. default is all surroundinx pixels', default=0, type=int)
parser.add_argument('-l', '--lease', help='lease_seconds for TaskQueue.poll. specify in seconds. default is 600sec', default=600, type=int)
args = parser.parse_args()

choose=args.choose
lease=args.lease

In [4]:
np.random.seed(123)
# queuepath = '/n/groups/htem/users/skuroda/nuclei_tasks'
queuepath = '../Output/nuclei_tasks'
# queuepath = '/n/groups/htem/users/skuroda/nuclei_output2'
outputpath = '../Output/'
size_xy = 160 # 160/(2**2)??
# 128x128x160 is small
# read csv
df = pd.read_csv('../Output/info_cellbody.csv', header=0)

In [5]:
# cv setting
seg = CloudVolume(auth.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False, cache=True, progress=False)

In [9]:
create_task_queue()

Inserting: 7it [00:00, 12.09it/s]

Done adding 7 tasks to queue at ../Output/nuclei_tasks


In [10]:
run_tasks_from_queue()

Working on tasks from filequeue "../Output/nuclei_tasks"
INFO Running FunctionTask(('__main__', 'task_cellbody2neuron'),[6],{},"e123c11b-372d-4aea-83c8-d09763369862")  (id: e123c11b-372d-4aea-83c8-d09763369862)


Inserting: 7it [00:14, 12.09it/s]/usr/local/pip-global/ipykernel_launcher.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Inserting: 7it [00:23,  3.39s/it]


INFO Deleting e123c11b-372d-4aea-83c8-d09763369862
INFO FunctionTask e123c11b-372d-4aea-83c8-d09763369862 succesfully executed in 29.74 sec.
INFO Running FunctionTask(('__main__', 'task_cellbody2neuron'),[3],{},"7a829302-077f-42af-8643-89c8bbfe859f")  (id: 7a829302-077f-42af-8643-89c8bbfe859f)
INFO Deleting 7a829302-077f-42af-8643-89c8bbfe859f
INFO FunctionTask 7a829302-077f-42af-8643-89c8bbfe859f succesfully executed in 32.52 sec.
INFO Running FunctionTask(('__main__', 'task_cellbody2neuron'),[0],{},"7a9515c5-4f42-4f1f-a898-1802881e51d5")  (id: 7a9515c5-4f42-4f1f-a898-1802881e51d5)
INFO Deleting 7a9515c5-4f42-4f1f-a898-1802881e51d5
INFO FunctionTask 7a9515c5-4f42-4f1f-a898-1802881e51d5 succesfully executed in 69.22 sec.
INFO Running FunctionTask(('__main__', 'task_cellbody2neuron'),[4],{},"c3e266fd-4413-4e3a-a466-dcc256411bb4")  (id: c3e266fd-4413-4e3a-a466-dcc256411bb4)
Interrupted. Exiting after this task completes. Press Ctrl-C again to exit now.
Failed, retrying


functions listed below

In [6]:
def vol_shift(input): # Although np.roll is fast, this is very slow since this overuse RAM
    # x plane
    x_p = np.roll(input, 1, axis=0)
    x_p[0,:,:] = 0
    x_n = np.roll(input, -1, axis=0)
    x_n[-1,:,:] = 0
    # y plane
    y_p = np.roll(input, 1, axis=1)
    y_p[:,0,:] = 0
    y_n = np.roll(input, -1, axis=1)
    y_n[:,-1,:] = 0
    # z plane
    z_p = np.roll(input, 1, axis=2)
    z_p[:,:,0] = 0
    z_n = np.roll(input, -1, axis=2)
    z_n[:,:,-1] = 0

    sum = x_p + x_n + y_p + y_n + z_p + z_n
    result = sum - input*6

    return result

In [7]:
# global variable is pt, segid, sizexy, choose

choose = 0

@queueable
def task_cellbody2neuron(i):
  try:
    cord_mip0 = df.iloc[i,0:3] #xyz coordinates
    cord_mip2 = cord_mip0.values # change coordination from mip0 to mip2
    cord_mip2[0]  = (cord_mip0.values[0] /(2**2))
    cord_mip2[1]  = (cord_mip0.values[1] /(2**2))
    cord_mip2 = cord_mip2.astype('int64')
    id = df.iloc[i,3] #segid

    if id == 0:
      A = np.append(cord_mip0.values, id).astype('int64')
      B = np.zeros(3, dtype = 'int64')
      output = np.append(A, B) #xyz, id, 0,0,0
    else:
      seg_nuc = seg.download_point(pt=cord_mip2, segids=id, size=[size_xy, size_xy, 160], coord_resolution=[17.2, 17.2, 45.0])
      # lowest resolution of seg is [17.2, 17.2, 45.0]
      vol_temp = seg_nuc[:,:,:]
      vol_temp[vol_temp>0] = 1 # change segID assigned to each cell body into 1
      vol = np.squeeze(vol_temp)

      filled = fill_voids.fill(vol, in_place=False) # fill the empty space with one 
      # ignore warning

      shifted = vol_shift(filled) # shift the volume
      shifted = shifted.astype('float32')
      shifted[shifted>0] = 1
      shifted[shifted<0] = 0

      location_one = np.argwhere(shifted == 1)
      len(location_one)

      if len(location_one):
        origin = seg_nuc.bounds.minpt # 3072,5248,1792
        parent_coordinates_mip2 = np.add(np.array(location_one), origin)
        parent_coordinates = parent_coordinates_mip2
        parent_coordinates[:,0]  = (parent_coordinates_mip2[:,0] * 2**2)
        parent_coordinates[:,1]  = (parent_coordinates_mip2[:,1] * 2**2)
        parent_coordinates = parent_coordinates.astype('int64')

        #random selection?
        if choose == 0:
          location_random = parent_coordinates
        else:
          index = np.random.choice(parent_coordinates.shape[0], size=choose, replace=False)
          location_random = parent_coordinates[index]

        # Lets get IDs using cell_body_coordinates
        parent_IDs = IDlook.segIDs_from_pts_cv(pts=location_random, cv=seg, progress=False) #mip0

        # save
        uniqueID, count = np.unique(parent_IDs, return_counts=True)
        unsorted_max_indices = np.argpartition(-count, 4)[:4]
        topIDs = uniqueID[unsorted_max_indices] # gives me top5 IDs
        topIDs2 = topIDs[~(topIDs == id)] # I hope this keeps order
        topIDs3 = topIDs2[~(topIDs2 == 0)] # no zero
        A = np.append(cord_mip0.values, id).astype('int64')
        B = topIDs3.astype('int64')[0:3]
        output = np.append(A, B) #top3
        
      else:
        A = np.append(cord_mip0.values, id).astype('int64')
        B = np.zeros(3, dtype = 'int64')
        output = np.append(A, B) #xyz, id, 0,0,0

      output_df = pd.DataFrame(columns=["x", "y", "z", "segIDs", "Parent1", "Parent2", "Parent3"])
      output_df.loc[0] = output
      name = str(i)
      output_df.to_csv(outputpath + '/' + 'cellbody_and_neuron_{}.csv'.format(name), index=False)

    seg.cache.flush()

  except Exception as e:
    name=str(i)
    with open(outputpath + '/' + '{}.log'.format(name), 'w') as logfile:
      print(e, file=logfile)

In [8]:
# task queue
# global variable is lease
 
def create_task_queue():
    tq = TaskQueue('fq://' + queuepath)
    tq.insert(( partial(task_cellbody2neuron, i) for i in range(len(df)) )) # NEW SCHOOL?
    print('Done adding {} tasks to queue at {}'.format(len(df), queuepath))


def run_tasks_from_queue():
    tq = TaskQueue('fq://' + queuepath)
    print('Working on tasks from filequeue "{}"'.format(queuepath))
    tq.poll(
        verbose=True, # prints progress
        lease_seconds=600,
        tally=True # makes tq.completed work, logs 1 byte per completed task
    )
    print('Done')